In [3]:
import pandas as pd 

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer


/Users/klvp919/.pyenv/versions/3.9.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/klvp919/.pyenv/versions/3.9.1/lib/python3.9/site-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/Users/klvp919/.pyenv/versions/3.9.1/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/t

In [ ]:

encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:

df = pd.read_csv('./data/top_rated_wines.csv')

## remove the empty values
df = df[df['variety'].notna()]
data = df.to_dict('records')
df